In [1]:
import pymysql
import pandas as pd
import numpy as np
import urllib
import os

from barycorrpy import utc_tdb

from astropy.time import Time
from astropy.coordinates import EarthLocation, SkyCoord
from astropy import units as u

In [11]:
%set_env BLUSR=interns
%set_env BLIP=104.154.94.28
%set_env BLPASS=FZtjWD3B4Yv5aq3m

env: BLUSR=interns
env: BLIP=104.154.94.28
env: BLPASS=FZtjWD3B4Yv5aq3m


In [12]:
# Get GBT TESS Targets os.environ['BLIP']

BLtargets = pymysql.connect(host=os.environ['BLIP'],user=os.environ['BLUSR'],
            password=os.environ['BLPASS'],database="BLtargets")

BLquery = """
SELECT *
FROM go_scans
WHERE target_name LIKE 'TIC%'
"""

go_scans = pd.read_sql(BLquery, BLtargets)

In [16]:
# Get full dataframe of TESS candidates

toiPath = os.path.join(os.getcwd(), 'TESS-toi.csv')
urllib.request.urlretrieve('https://exofop.ipac.caltech.edu/tess/download_toi.php?sort=toi&output=csv', toiPath)
TESStoi = pd.read_csv(toiPath)

4074

In [13]:
go_scans['TIC ID'] = go_scans.target_name.apply(lambda v : int(v[3:]))
gbt = go_scans[['TIC ID', 'target_name', 'utc_observed', 'session']]

In [14]:
inTransitID = []
inTransitSession = []
gbtloc = EarthLocation.of_site('Green Bank Telescope')

for ticid in gbt['TIC ID'].unique():
    
    gbtInfo = gbt.loc[gbt['TIC ID'] == ticid]
    TESSinfo = TESStoi.loc[TESStoi['TIC ID'] == ticid]
    
    epoch = TESSinfo['Epoch (BJD)'].to_numpy()[0] # BJD
    period = TESSinfo['Period (days)'].to_numpy()[0]
    transitLength = TESSinfo['Duration (hours)'].to_numpy()[0]
    dist = TESSinfo['Stellar Distance (pc)'].to_numpy()[0]
    PMRA = float(TESSinfo['PM RA (mas/yr)'].to_numpy()[0])
    PMdec = float(TESSinfo['PM Dec (mas/yr)'].to_numpy()[0])
    
    ra = TESSinfo['RA'].to_numpy()[0]
    dec = TESSinfo['Dec'].to_numpy()[0]
    coords = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame='icrs')
    
    obsTime = gbtInfo['utc_observed'].to_numpy()
    session = gbtInfo['session'].to_numpy()
    
    parallax = (1/dist) * 10**(-3) # units of mas
    
    
        
    # Convert Observed time to BJD
    tUTC = Time(obsTime, location=gbtloc)
    tbjd = utc_tdb.JDUTC_to_BJDTDB(tUTC, ra=float(coords.to_string().split()[0]), 
                                       dec=float(coords.to_string().split()[1]), 
                                       pmra=PMRA, pmdec=PMdec, 
                                       px= parallax, obsname='Green Bank Telescope')[0]

    tbjd = tbjd[np.isfinite(tbjd)]

    # Does the GBT observation occur during a transit?
    if len(obsTime) >= 3 and period != 0: # Check for at least 3 obs and remove 'nan' entries
        
        tt = transitLength/24/2 # half transit time and convert to days
        
        # Create list of center transit times for each ticid
        
        for ii, obst in enumerate(tbjd):
            diff = np.abs(obst-epoch)
            numRot = int(np.ceil(diff/period))
            
            centerTransitTimes = []
            t = epoch
            for i in range(numRot):
                centerTransitTimes.append(t)
                if obst < epoch:
                    t-=period
                else:
                    t+=period
            
            if np.abs(centerTransitTimes[-1] - obst) > period:
                print(f'{centerTransitTimes[-1] - obst} is outside one period')  
                
            epochf = centerTransitTimes[-1]
            startTransit = epochf - tt
            endTransit = epochf + tt
            
            if obst > startTransit and obst < endTransit:
                inTransitID.append(ticid)
                inTransitSession.append(session[ii])

In [15]:
outFrame = []

for tic, sess in zip(inTransitID, inTransitSession):

    mask = (go_scans['TIC ID'] == tic) & (go_scans['session'] == sess)
    
    if len(mask[mask==True]) >= 3:
        outFrame.append(go_scans[mask])

targets = pd.concat(outFrame).drop_duplicates()

outFilePath = os.path.join(os.getcwd(), 'TESStargets.csv')
    
targets.to_csv(outFilePath)

In [16]:
print(f'{len(targets.target_name.unique())} TESS targets found in transit')

61 TESS targets found in transit


In [17]:
perRecovered = len(targets.target_name.unique()) / len(go_scans['TIC ID'].unique()) * 100

estPer = (np.mean(TESStoi['Duration (hours)'])/24) / np.mean(TESStoi['Period (days)']) * 100

perRecovered, estPer

(6.192893401015229, 1.5975380571030842)